# 경사 하강법을 이용한 얕은 신경망 학습

In [55]:
import tensorflow as tf
import numpy as np

### 하이퍼 파라미터 설정

In [56]:
EPOCHS = 100

### 네트워크 구조 정의
- 얕은 신경망
- 입력 계층 : 2, 은닉 계층 : 128(sigmoid activation), 출력 계층 : 10 (Softmax activation)

In [57]:
class MyModel(tf.keras.Model) :
    def __init__(self) :
        super(MyModel, self).__init__()
        self.d1 = tf.keras.layers.Dense(128, input_dim=2, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, input_dim=2, activation='softmax')
        
    def call(self, x, training=None, mask=None) :
        x = self.d1(x)
        return self.d2(x)

### 학습 루프 정의

In [58]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric) :
    with tf.GradientTape() as tape :
        predictions = model(inputs)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables) #df(x)/dx
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_metric(labels, predictions)
    
# @tf.function
# def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
#     with tf.GradientTape() as tape:
#         predictions = model(inputs)
#         loss = loss_object(labels, predictions)
#     gradients = tape.gradient(loss, model.trainable_variables) # df(x)/dx
    
#     optimizer.apply_gradients(zip(gradients, model.trainable_variables))
#     train_loss(loss)
#     train_metric(labels, predictions)

### 데이터셋 생성, 전처리

In [59]:
np.random.seed(0)

pts = list()
labels = list()
center_pts = np.random.uniform(-8.0, 8.0, (10,2))
for label, center_pt in enumerate(center_pts) :
    for _ in range(100) :
        pts.append(center_pt + np.random.rand(*center_pt.shape))
        labels.append(label)
        
pts = np.stack(pts, axis=0).astype(np.float32)
labels = np.stack(labels, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)

### 모델 생성

In [60]:
model = MyModel()

### 손실 함수 및 최적화 알고리즘 설정
- CrossEntropy, Adam Optimizer

In [61]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

### 평가 지표 설정
- Accuracy

In [62]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

### 학습 루프

In [63]:
for epoch in range(EPOCHS) :
    for x, label in train_ds :
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)
        
    template = 'Epoch : {}, Loss : {}, Accuracy : {}'
    print(template.format(epoch+1, train_loss.result(), train_accuracy.result()*100))
        
    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch : 1, Loss : 2.2030646800994873, Accuracy : 30.89999771118164
Epoch : 2, Loss : 1.837672233581543, Accuracy : 46.39999771118164
Epoch : 3, Loss : 1.5876739025115967, Accuracy : 57.5
Epoch : 4, Loss : 1.4245270490646362, Accuracy : 65.5
Epoch : 5, Loss : 1.3007673025131226, Accuracy : 73.0999984741211
Epoch : 6, Loss : 1.1895177364349365, Accuracy : 89.20000457763672
Epoch : 7, Loss : 1.089059829711914, Accuracy : 87.0
Epoch : 8, Loss : 1.0112677812576294, Accuracy : 90.0
Epoch : 9, Loss : 0.9473810195922852, Accuracy : 95.80000305175781
Epoch : 10, Loss : 0.8717984557151794, Accuracy : 97.0999984741211
Epoch : 11, Loss : 0.8181965947151184, Accuracy : 95.0999984741211
Epoch : 12, Loss : 0.7685077786445618, Accuracy : 98.5
Epoch : 13, Loss : 0.7191131711006165, Accuracy : 98.9000015258789
Epoch : 14, Loss : 0.6813620924949646, Accuracy : 99.9000015258789
Epoch : 15, Loss : 0.6363794803619385, Accuracy : 100.0
Epoch : 16, Loss : 0.5943858623504639, Accuracy : 100.0
Epoch : 17, Loss 

### 데이터셋

In [64]:
np.savez_compressed('ch2_dataset.npz', inputs=pts, labels=labels)

w_h, b_h = model.d1.get_weights() 
w_o, b_o = model.d2.get_weights()
w_h = w_h.T
w_o = w_o.T
np.savez_compressed('ch2_parameters.npz', w_h=w_h, b_h=b_h
                   , w_o = w_o, b_o = b_o)